In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Upload Data and Import Libraries

In [ ]:
from zipfile import ZipFile 
# specifying the zip file name 
file_name = "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/data.zip"
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zi: 
    # extracting all the files 
    print('Extracting all the files now...') 
    zi.extractall() 
    print('Done!')

Extracting all the files now...
Done!


In [ ]:
!cp "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/Train.csv" . 
!cp "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/SampleSubmission.csv" .

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=5e3b0ae8deb8fc7334e321e697264a509c60ed43e9252fb71f6df2370aec912b
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision import transforms
import torch
import torchvision
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import train_test_split
import albumentations
from albumentations import torch as AT
import cv2

In [ ]:
import random
import numpy as np
SEED_VAL  = 1000
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True

# Preparing Dataset 

In [ ]:
def default_image_loader(path):
    return Image.open(path).convert('RGB')
def cv_reader(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

class ImageDataset(Dataset):
    def __init__(self, data_path, df, transform=None,mode='train'):
        self.df = df
        self.loader = cv_reader
        self.transform = transform
        self.dir = data_path
        self.mode = mode

    def __getitem__(self, index):
        image_name = self.df.image_name[index]
        image = self.loader(os.path.join(self.dir, image_name+'.jpeg'))
        if self.transform is not None:
          image = self.transform(image=image)
          image = image['image']
        
        

        if self.mode == 'train':
            return {'image' : torch.tensor(image,dtype=torch.float), 
                'label' : torch.tensor(np.argmax(self.df.iloc[index,3:].values)) }
            
        return {'image' : torch.tensor(image,dtype=torch.float), 
}            
        
    
    def __len__(self):
        return self.df.shape[0]

In [ ]:
class Net(nn.Module):
    def __init__(self,name):
        super(Net, self).__init__()
        self.name  = name
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=7, bias=True)
        elif name == 'b1':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b1')
          self.arch._fc = nn.Linear(in_features=1280, out_features=7, bias=True)
        elif name == 'b2':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b2')
          self.arch._fc = nn.Linear(in_features=1408, out_features=7, bias=True)
        elif name =='b3':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b3')
          self.arch._fc = nn.Linear(in_features=1536, out_features=7, bias=True)
        elif name =='b4':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b4')
          self.arch._fc = nn.Linear(in_features=1792, out_features=7, bias=True)
        elif name =='b5':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b5')
          self.arch._fc = nn.Linear(in_features=2048, out_features=7, bias=True)
        elif name =='b6':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b6')
          self.arch._fc = nn.Linear(in_features=2304, out_features=7, bias=True)
        elif name =='b7':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b7')
          self.arch._fc = nn.Linear(in_features=2560, out_features=7, bias=True)
        elif name == 'densenet121':
          self.arch = models.densenet121(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'densenet169':
          self.arch = models.densenet169(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnet50':
          self.arch = models.resnet50(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnet101':
          self.arch = models.resnet101(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnet152':
          self.arch = models.resnet152(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnet34':
          self.arch = models.resnet34(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnext101':
          self.arch = models.resnext101_32x8d(pretrained=True)
          self.arch.drop_out = nn.Dropout(p=0.2)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name == 'resnext50':
          self.arch = models.resnext50_32x4d(pretrained=True)
          self.arch.drop_out = nn.Dropout(p=0.2)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,7,bias=True)
        elif name =='rexnetv1':
            model = rexnetv1.ReXNetV1(width_mult=1.0)
            model.output.conv2D = nn.Conv2d(1280, 1, kernel_size=(1, 1), stride=(1, 1))
          
    def forward(self, x):
        """
        """
        x = self.arch(x)
        return x

In [ ]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def loss_ce(outputs,targets):
  criterion = nn.CrossEntropyLoss()
  loss = criterion(outputs,targets)
  return loss

In [ ]:
def train_fn(train_data_loader,model,optimizer,device,scheduler = None):
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(train_data_loader, total=len(train_data_loader))
  tot_loss = 0
  for bi,d in enumerate(tk0):
    images = d['image']
    labels = d['label']

    #send them to device 
    images = images.to(device,dtype=torch.float)
    labels = labels.to(device,dtype=torch.long)
    optimizer.zero_grad()

    outputs  = model(images)
    

    if LOSS == 'CE':
      loss = loss_ce(outputs,labels)
      loss.backward()
      optimizer.step()
      tot_loss = tot_loss + loss.item()
      losses.update(loss.item(), labels.size(0))
      tk0.set_postfix(loss_ce=losses.avg)

    if scheduler is not None:
      scheduler.step()
  return losses.avg
  

In [ ]:
def eval_fn(valid_data_loader,model,device):
  model.eval()
  tot_loss = 0
  final_outputs = []
  final_targets = []
  with torch.no_grad():
    for bi,d in enumerate(valid_data_loader):

      images = d['image']
      labels = d['label']

      #send them to device 
      images = images.to(device,dtype=torch.float)
      labels = labels.to(device,dtype=torch.long)
      outputs  = model(images)

      if LOSS == 'CE':
        loss = loss_ce(outputs,labels)
        tot_loss = tot_loss + loss.item()
        
      final_outputs.extend(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
      final_targets.extend(labels.cpu().numpy())
    reg_outputs = []
    for i in final_outputs:
      result = np.sum(i * [1,2,3,4,5,6,7])
      reg_outputs.append(result)


    final_outputs = np.argmax(final_outputs,axis=1) 
    mean_loss_val  = tot_loss/len(valid_data_loader)
    acc_score = sklearn.metrics.accuracy_score(final_targets,final_outputs)
    rmse_score = np.sqrt(((np.subtract(np.array(final_targets)+1,reg_outputs))**2).mean())
    print(f"Validation loss {LOSS} for this epoch: ",mean_loss_val)
    print('Validation accuracy for this epoch',acc_score)
    print('Validation rmse for this epoch',rmse_score)
  return rmse_score


In [ ]:
input_shape = [128,256]
def get_transforms():

  train_transform = albumentations.Compose([
      albumentations.Resize(input_shape[0],input_shape[1]),
      albumentations.Normalize(),
      AT.ToTensor()
      ])
  val_transform = albumentations.Compose([
      albumentations.Resize(input_shape[0],input_shape[1]),
      albumentations.Normalize(),
      AT.ToTensor()
      ])

  
  return train_transform,val_transform

## Stratfied Kfolds

In [ ]:
NAME = 'densenet169'
EPOCHS = 20
TRAIN_BATCH_SIZE = 64
BOOL_CROP = False
LOSS = 'CE'
LR = 2e-4
skf = StratifiedKFold(n_splits=10,random_state=SEED_VAL)
all_rmse_scores = []
def run_folds():
  seed_all(SEED_VAL)
  train_transform,val_transform = get_transforms()

  train = pd.read_csv('/content/Train.csv')
  cols_labels = pd.get_dummies(train['growth_stage'] ,prefix='groth_stage')
  train = pd.concat([train,cols_labels],axis=1 ) 
  train = train[train['label_quality']==2]
  train.reset_index(drop=True,inplace=True)
  train = train.rename(columns = {'UID':'image_name','growth_stage':'target'})
  DF = pd.DataFrame()
  for i,(train_index,val_index) in enumerate(skf.split(train,y=train.target)):
    print(f"#########################  Fold {i+1}/{skf.n_splits}  #########################")
    train_df , valid_df = train.iloc[train_index,:],train.iloc[val_index,:]
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)
    train_dataset = ImageDataset('Images',train_df,train_transform)
    valid_dataset = ImageDataset('Images',valid_df,val_transform)
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=TRAIN_BATCH_SIZE)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=16)
    device = torch.device("cuda")
    model = Net(NAME)
    model.to(device)
    num_train_steps = int(len(train_df) /TRAIN_BATCH_SIZE * EPOCHS )
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #scheduler = MultiStepLR(optimizer, milestones=[2,5,10], gamma=0.1)
    best_rmse = 1500
    
    for epoch in range(EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      rmse_train = train_fn(train_data_loader, model, optimizer, device,scheduler=None)
      rmse_val = eval_fn(valid_data_loader ,model, device)
      if rmse_val<best_rmse:
        best_rmse = rmse_val 
        torch.save(model.state_dict(),f"best_model_{i}") 
    print(f'best VAL_RMSE for fold {i+1}: ',best_rmse)
    DF = DF.append({'FODL':1+i,'BEST_VAL_RMSE':best_rmse},ignore_index=True)
    all_rmse_scores.append(best_rmse)
  print(f"MEAN RMSE: {np.mean(all_rmse_scores)}")
  return DF


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
kfold_results = run_folds()

#########################  Fold 1/10  #########################


Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth



----------------EPOCH 1---------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...


Validation loss CE for this epoch:  0.7064522938906319
Validation accuracy for this epoch 0.8105263157894737
Validation rmse for this epoch 0.5406238371699655
----------------EPOCH 17---------------------



Validation loss CE for this epoch:  0.6150538886690305
Validation accuracy for this epoch 0.8385964912280702
Validation rmse for this epoch 0.45107611393831354
----------------EPOCH 18---------------------



Validation loss CE for this epoch:  0.762668633626567
Validation accuracy for this epoch 0.8105263157894737
Validation rmse for this epoch 0.5055998731644002
----------------EPOCH 19---------------------



Validation loss CE for this epoch:  0.6774896366728677
Validation accuracy for this epoch 0.8070175438596491
Validation rmse for this epoch 0.5330306394271165
----------------EPOCH 20---------------------



Validation loss CE for this epoch:  0.8313442365162902
Validation accuracy for this epoch 0.8210526315789474
Validation rmse for this epoch 0.5634137910889041
best VAL_RMSE for fold 10:  0.45107611393831354
MEAN RMSE: 0.4539599068830988


In [ ]:
kfold_results

,BEST_VAL_RMSE,FODL
0,0.412996,1.0
1,0.429595,2.0
2,0.404455,3.0
3,0.479604,4.0
4,0.406824,5.0
5,0.465588,6.0
6,0.448129,7.0
7,0.549803,8.0
8,0.491530,9.0
9,0.451076,10.0


# Predicting

In [ ]:
test_transform = albumentations.Compose([
      albumentations.Resize(input_shape[0],input_shape[1]),
      albumentations.Normalize(),
      AT.ToTensor()
      ])
  
test = pd.read_csv('SampleSubmission.csv')
test.rename(columns={'UID':'image_name'},inplace=True)
test_dataset = ImageDataset('Images',test,test_transform,mode='test')
test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=32)

## KFOLDS

In [ ]:
all_folds = []
for i in range(skf.n_splits):
  best_model = Net(NAME)
  best_model.load_state_dict(torch.load(f'best_model_{i}'))
  best_model.to(device)
  best_model.eval()
  final_outputs = []
  with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi,d in enumerate(tk0):
      images = d['image']
      #send them to device 
      images = images.to(device,dtype=torch.float)
      outputs  = best_model(images)
      final_outputs.extend(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
  reg_outputs = []
  for i in final_outputs:
    result = np.sum(i*[1,2,3,4,5,6,7])
    reg_outputs.append(result)
    
  all_folds.append(reg_outputs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


In [ ]:
ss = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
ss['growth_stage'] = np.mean(all_folds,axis=0)
ss['growth_stage'] = ss['growth_stage'].clip(1,7)
ss.head()

,UID,growth_stage
0,F3LbWkZq,4.769467
1,6P4wLVzI,3.974977
2,Tn6HIrdS,2.997663
3,EYM5R1TK,4.316235
4,fkcevlA0,5.569253


In [ ]:
ss.growth_stage.max(),ss.growth_stage.min()

(6.999707712381524, 2.000110476591014)

In [ ]:
ss.to_csv(f'{NAME}_highQ_floats_Folds={skf.n_splits}input_shape=[{input_shape[0]}X{input_shape[1]}]_lr={LR}_epochs={EPOCHS}_bs={TRAIN_BATCH_SIZE}_dr=0.2Classifi.csv',index=False)

In [ ]:
ss.head()

,UID,growth_stage
0,F3LbWkZq,4.769467
1,6P4wLVzI,3.974977
2,Tn6HIrdS,2.997663
3,EYM5R1TK,4.316235
4,fkcevlA0,5.569253
